In [11]:
!pip install networkx
import networkx as nx
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd



---

## Função para plotar grafos no plano cartesiano 


In [12]:
def show_graph(graph):

  pos = nx.get_node_attributes(graph, "pos")
  fig, ax = plt.subplots()

  for i in graph.nodes():
    ax.annotate(i, (graph.nodes[i]['pos'][0], graph.nodes[i]['pos'][1]), fontsize=15)
  
  ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
  nx.draw_networkx_edges(graph, pos, alpha=0.4, ax=ax)
  nx.draw_networkx_nodes(graph, pos, node_size=100, cmap=plt.cm.Reds_r,)
  
  plt.show()



## Twice Around The Tree

In [13]:
def get_cost(graph, order):
  cost = 0
  for i in range(0, len(order) - 1):   
    cost += graph[order[i]][order[i+1]]['weight']

  return cost + graph[order[0]][order[len(order) - 1]]['weight']

In [14]:
def twice_around_the_tree(graph):
  mst = nx.minimum_spanning_tree(graph, weight = 'weight', algorithm="prim")
  order = list(nx.dfs_preorder_nodes(mst, 0))
  cost = get_cost(graph, order)
  
  return cost


## Algoritmo de Christofides

In [15]:
def christofides(graph):

  mst = nx.minimum_spanning_tree(graph, weight = 'weight', algorithm="prim") # Create a minimum spanning tree T of G.

  i = graph.subgraph(nodes_odd_degree) #  I is the induced subgraph given by the vertices from O
  matching = nx.min_weight_matching(i, maxcardinality  = True) # Find a minimum-weight perfect matching M in I
  for e in matching:
    weight = graph[e[0]][e[1]]['weight']
    
  h = nx.MultiGraph(mst)
  h.add_edges_from(matching) # Combine the edges of M and T to form a connected multigraph H in which each vertex has even degree.
  
  order = list(nx.dfs_preorder_nodes(h, 0))
  cost = get_cost(graph, order)
  
  return cost

## Branch and Bound


In [16]:
def bnb_recursive(graph, cbound, cweight, level, cpath, closest, visited, res):
  if level == graph.number_of_nodes():
    if cpath[level - 1] != cpath[0]:
      cres = cweight + graph[cpath[level-1]][cpath[0]]['weight']
      if cres < res: res = cres
    return res

  for v in graph.nodes():
    if cpath[level-1] != v and visited[v] == False:

      temp = cbound;
      cweight += graph[cpath[level-1]][v]['weight'];
  
      if level == 1: cbound -= ((closest[cpath[level-1]][0] + closest[v][0])/2)
      else: cbound -= ((closest[cpath[level-1]][1] + closest[v][0])/2)
      
      if (cbound + cweight) < res:
        cpath[level], visited[v] = v, True
        res = bnb_recursive(graph, cbound, cweight, level+1, cpath, closest, visited, res)
      
      cweight -= graph[cpath[level-1]][v]['weight']
      cbound = temp;
  
      visited = [False for i in graph.nodes()]
      for i in range(0,level): visited[cpath[i]] = True;
          
  return res

def branch_and_bound(graph):
  cbound, closest = 0, []
  cpath, visited = [-1 for i in range(0, graph.number_of_nodes()+1)], [False for i in graph.nodes()]
  visited[0], cpath[0] = True, 0

  for v in graph.nodes():
    min_weight_neighbors = sorted(graph[v].items(), key=lambda e: e[1]["weight"] if e[1]["weight"] != 0  else 1000000000)[:2]     
    closest.append([min_weight_neighbors[0][1]['weight'], min_weight_neighbors[1][1]['weight']])
    cbound += (min_weight_neighbors[0][1]['weight']+ min_weight_neighbors[1][1]['weight'])/2

  return bnb_recursive(graph, cbound, 0, 1, cpath, closest, visited,  np.Inf)

## Gerador do grafo geométrico 

In [17]:
def distance(v1, v2, metric):
  if metric == 'euclidean':
    return ((v1[0] - v2[0])**2 + (v1[1] - v2[1])**2)**(1/2)
  else:
    return abs(v1[0] - v2[0]) + abs(v1[1] - v2[1])

def generate_graph(metric, amount_of_nodes):

  graph = nx.soft_random_geometric_graph(n = amount_of_nodes, radius  = 10, dim = 2, p_dist = lambda dist: 1, seed = 5)
  for i in graph.nodes():
    for j in graph.nodes():
      if i != j:
        dist = distance(graph.nodes[i]['pos'], graph.nodes[j]['pos'], metric)
        graph.add_edge(i, j, weight = dist)
      
  return graph

## Gerador de Instâncias

In [18]:
 def instance_maker(algorithm): 
  df_cost, df_time = {}, {}

  for i in range (4, 11):
    graph_euclidean, graph_manhattan = generate_graph('euclidean', 2**i), generate_graph('manhattan', 2**i)

    if algorithm == 'Twice Around The Tree':
      init = time.time()
      cost_e = twice_around_the_tree(graph_euclidean) 
      period_e = time.time() - init                  
      init = time.time()
      cost_m = twice_around_the_tree(graph_manhattan) 
      period_m = time.time() - init      
    
    elif algorithm == 'Christofides':
      init = time.time()
      cost_e = christofides(graph_euclidean) 
      period_e = time.time() - init                  
      init = time.time()
      cost_m = christofides(graph_manhattan) 
      period_m = time.time() - init

    else:
      if i >= 5:
        df_cost[2**i], df_time[2**i] = ["NA", "NA"], ["NA", "NA"]  
        continue
      else:   
        init = time.time()
        cost_e = branch_and_bound(graph_euclidean) 
        period_e = time.time() - init                  
        init = time.time()
        cost_m = branch_and_bound(graph_manhattan) 
        period_m = time.time() - init             

    df_cost[2**i], df_time[2**i] = ["{:.5f}".format(cost_e), "{:.5f}".format(cost_m)], ["{:.5f}".format(period_e), "{:.5f}".format(period_m)]  

    print('\n', algorithm, 2**i, 'nodes:')
    print('Euclidean distance:', cost_e)
    print('Manhattan distance:', cost_m)
  
  return df_cost, df_time

In [10]:
def main():  
  
  tat_cost, tat_time = instance_maker('Twice Around The Tree')
  print('----------------------------------------------------------------------------------------')
  ch_cost, ch_time = instance_maker('Christofides')
  print('----------------------------------------------------------------------------------------')
  bab_cost, bab_time = instance_maker('Branch and Bound')
  print('----------------------------------------------------------------------------------------')

  cost = pd.DataFrame({'Twice Around The Tree':tat_cost, 'Christofides':ch_cost, 'Branch and Bound':bab_cost})
  time = pd.DataFrame({'Twice Around The Tree':tat_time, 'Christofides':ch_time, 'Branch and Bound':bab_time})

  cost.to_csv("costs.csv")
  time.to_csv("times.csv")

main()


 Twice Around The Tree 16 nodes:
Euclidean distance: 5.11529544230702
Manhattan distance: 4.766660928684083

 Twice Around The Tree 32 nodes:
Euclidean distance: 6.788482692112228
Manhattan distance: 8.957591059150808

 Twice Around The Tree 64 nodes:
Euclidean distance: 7.921827086094279
Manhattan distance: 9.907637981321063

 Twice Around The Tree 128 nodes:
Euclidean distance: 12.262415093800096
Manhattan distance: 15.078299153052784

 Twice Around The Tree 256 nodes:
Euclidean distance: 15.943903478829085
Manhattan distance: 20.23064616105926

 Twice Around The Tree 512 nodes:
Euclidean distance: 22.3634884183945
Manhattan distance: 28.350455920208862

 Twice Around The Tree 1024 nodes:
Euclidean distance: 31.451981064760627
Manhattan distance: 39.23458468781248
----------------------------------------------------------------------------------------

 Christofides 16 nodes:
Euclidean distance: 4.120916595632929
Manhattan distance: 5.426766867882269

 Christofides 32 nodes:
Euclide